## Eksperimen 1: Visual Problem Generation (VLLM)

Generate soal visual dengan VLLM.

In [161]:
print("Setup VLLM experiment...")

Setup VLLM experiment...


In [162]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install -q openai requests

from openai import OpenAI
import json, random, time

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
resp = requests.get("http://localhost:8000/v1/models").json()
resp


{'object': 'list',
 'data': [{'id': 'Qwen/Qwen1.5-0.5B-Chat',
   'object': 'model',
   'created': 1769332595,
   'owned_by': 'vllm',
   'root': 'Qwen/Qwen1.5-0.5B-Chat',
   'parent': None,
   'max_model_len': 1024,
   'permission': [{'id': 'modelperm-996594e4a29174aa',
     'object': 'model_permission',
     'created': 1769332595,
     'allow_create_engine': False,
     'allow_sampling': True,
     'allow_logprobs': True,
     'allow_search_indices': False,
     'allow_view': True,
     'allow_fine_tuning': False,
     'organization': '*',
     'group': None,
     'is_blocking': False}]}]}

In [ ]:
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="EMPTY"
)

In [ ]:
MODEL_NAME = "Qwen/Qwen1.5-1.8B-Chat"
OUTPUT_FILE = "bank_soal_algo.json"

In [ ]:
TOPICS = {
    "Percabangan if": ["ganjil genap", "nilai lulus", "diskon"],
    "Loop for": ["jumlah deret", "perkalian", "tabel perkalian"],
    "Loop while": ["input sampai 0", "validasi input"],
    "Array dasar": ["jumlah array", "rata-rata array"],
    "Mencari maksimum": ["maks 3 angka", "maks array"],
    "Faktorial": ["faktorial n"],
    "Bilangan prima": ["cek prima", "prima 1 sampai n"],
    "Sorting sederhana": ["bubble sort"],
    "String dasar": ["hitung vokal", "palindrome"]
}

In [ ]:
def build_prompt(topic, subtopic, seed):
    return f"""
KAMU ADALAH DOSEN ALGORITMA PEMROGRAMAN.

Buat SATU soal koding baru.

Topik: {topic}
Subtopik: {subtopic}
Seed variasi: {seed}

Output HARUS JSON valid dengan field:
- judul
- deskripsi
- pseudocode
- template_program
- kunci_jawaban
- mermaid

ATURAN:
- Soal cerita
- Bahasa Indonesia
- Level dasar
- Berbeda dari soal lain
- Template Python
- Output hanya JSON

Pseudocode format:

====
program ...
algoritma:
    ...
====

JANGAN beri teks di luar JSON.
"""

In [ ]:
def generate_one(topic, subtopic, seed, retry=3):
    for attempt in range(retry):
        try:
            prompt = build_prompt(topic, subtopic, seed)

            resp = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": "Output ONLY valid JSON."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=900,
                response_format={"type": "json_object"}
            )

            raw = resp.choices[0].message.content
            data = json.loads(raw)

            return data

        except Exception as e:
            print("Retry...", attempt+1)

    return None

In [ ]:
def generate_bank(n_per_subtopic=5):
    bank = []

    for topic, subs in TOPICS.items():
        for sub in subs:
            for i in range(n_per_subtopic):

                seed = random.randint(0, 999999)

                data = generate_one(topic, sub, seed)

                if data:
                    bank.append(data)
                    print("✔", topic, "-", sub, "-", i+1)
                else:
                    print("gagal:", topic, sub)

    return bank

In [ ]:
def save_bank(bank):
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(bank, f, indent=2, ensure_ascii=False)

    print("\nBank soal tersimpan:", OUTPUT_FILE)


Waktu inferensi: 1.0912373065948486

PARSED JSON:
 {
    "soal": "Sebuah persegi memiliki sisi 8 cm. Pada salah satu sisinya ditempel segitiga siku-siku dengan alas 8 cm dan tinggi 6 cm. Hitung luas total bangun datar gabungan tersebut.",
    "solution": "Luas persegi = 8 × 8 = 64 cm². Luas segitiga = 1/2 × 8 × 6 = 24 cm². Jadi luas total = 88 cm²."
}


In [ ]:
if __name__ == "__main__":

    JUMLAH_PER_SUBTOPIK = 10

    start = time.time()

    bank = generate_bank(JUMLAH_PER_SUBTOPIK)

    save_bank(bank)

    end = time.time()

    print("\nTotal soal:", len(bank))
    print("Waktu:", round(end-start, 2), "detik")